In [1]:
using LinearAlgebra
using ProgressMeter
using StatsBase
using Plots
using FisherGillespie
using CurveFit

import Random
Random.seed!(1234)

function generate_annihilation(N)
    matrix = complex(float(zeros(N,N)))
    for n in 2:N
        n_ket = complex(float(zeros(N)))
        n_ket[n] = 1
        n_bra = complex(float(zeros(N)))
        n_bra[n-1] = 1
        matrix += sqrt(n-1) * n_bra * n_ket'
    end
    return matrix
end;

In [2]:
v_slopes = []
θ_min = 0.
θ_max = π
θ_step = π/20.

θ_range = θ_min:θ_step:θ_max

for θ in θ_range
    # Angles defining the initial state of the incoming atoms.
    ϕ = 0.
    # Translated into coefficients for simplicity.
    α = cos(θ)
    β = sin(θ) * exp(1im * ϕ)
    
    # Model parameters.
    r = 0.1 # Injection rate for new atoms.
    g = 1. # Coupling strength between atom and field.
    τ = 1. # Flight time of atom in cavity.
    γ = 0.1 # Leakage rate of cavity.
    nth = 0.1 # Average number of photons in environment.

    # Simulation parameters.
    space_cutoff = 5
    number_trajectories = 100
    dt = 0.001
    t_final = 100.
    t_range = 0:dt:t_final
    dg = 0.01

    # Initial state: ground state of the cavity.
    ρ0 = complex(zeros((space_cutoff, space_cutoff)))
    ρ0[1,1] = 1
    ψ0 = complex(zeros(space_cutoff))
    ψ0[1] = 1.

    # Useful operators.
    a = generate_annihilation(space_cutoff)
    ide = I(space_cutoff)

    # Hamiltonian for the cavity state.
    H = a' * a

    R = sqrt(a' * a + ide)
    Q = sqrt(a' * a)

    # Jump operators.
    M_ae = sqrt(r) * (α * cos(g * τ * R) - 1im * β * sin(g * τ * R)/R * a) # Atom detected in excited state.
    M_ag = sqrt(r) * (α * a' * sin(g * τ * R)/R + 1im * β * cos(g * τ * Q)) # Atom detected in ground state.
    M_po = sqrt(γ * (nth + 1)) * a # Outcoming photon detected.
    M_pi = sqrt(γ * nth) * a'; # Incoming photon detected.

    # Plus-displaced operators.
    Hp = H
    Mp_ae = sqrt(r) * (α * cos((g + dg) * τ * R) - 1im * β * sin(g * τ * R)/R * a) # Atom detected in excited state.
    Mp_ag = sqrt(r) * (α * a' * sin((g + dg) * τ * R)/R + 1im * β * cos(g * τ * Q)) # Atom detected in ground state.
    Mp_po = sqrt(γ * (nth + 1)) * a # Outcoming photon detected.
    Mp_pi = sqrt(γ * nth) * a'; # Incoming photon detected.

    # Minus-displaced operators.
    Hm = H
    Mm_ae = sqrt(r) * (α * cos((g - dg) * τ * R) - 1im * β * sin(g * τ * R)/R * a) # Atom detected in excited state.
    Mm_ag = sqrt(r) * (α * a' * sin((g - dg) * τ * R)/R + 1im * β * cos(g * τ * Q)) # Atom detected in ground state.
    Mm_po = sqrt(γ * (nth + 1)) * a # Outcoming photon detected.
    Mm_pi = sqrt(γ * nth) * a'; # Incoming photon detected.

    M_l = [M_ae, M_ag, M_po, M_pi]
    Mp_l = [Mp_ae, Mp_ag, Mp_po, Mp_pi]
    Mm_l = [Mm_ae, Mm_ag, Mm_po, Mm_pi]

    S_l = Matrix{ComplexF64}[]
    Sp_l = Matrix{ComplexF64}[]
    Sm_l = Matrix{ComplexF64}[]

    trajectories_results, V, Vdot, t_range = FisherGillespie.gillespie_fisher(H, Hp, Hm, M_l, Mp_l, Mm_l, dg, ψ0, t_final, dt, number_trajectories);
    average_fisher = zeros(length(t_range))
    @showprogress "Filling trajectory..." for n_trajectory in eachindex(trajectories_results)
        v_fisher = FisherGillespie.fisher_at_time_on_trajectory(t_range, t_range, V, Vdot, trajectories_results[n_trajectory], ψ0)
        average_fisher += v_fisher / number_trajectories
    end
    a, m = linear_fit(t_range, average_fisher)
    push!(v_slopes, m)
end

Fisher-Gillespie evolution... 100%|██████████████████████| Time: 0:01:03
Filling trajectory... 100%|██████████████████████████████| Time: 0:00:36
Fisher-Gillespie evolution... 100%|██████████████████████| Time: 0:00:59
Filling trajectory... 100%|██████████████████████████████| Time: 0:00:42
Fisher-Gillespie evolution... 100%|██████████████████████| Time: 0:01:15
Filling trajectory... 100%|██████████████████████████████| Time: 0:00:39
Fisher-Gillespie evolution... 100%|██████████████████████| Time: 0:00:55
Filling trajectory... 100%|██████████████████████████████| Time: 0:00:34
Fisher-Gillespie evolution... 100%|██████████████████████| Time: 0:01:11
Filling trajectory... 100%|██████████████████████████████| Time: 0:00:51
Fisher-Gillespie evolution... 100%|██████████████████████| Time: 0:00:58
Filling trajectory... 100%|██████████████████████████████| Time: 0:00:49
Fisher-Gillespie evolution... 100%|██████████████████████| Time: 0:01:08
Filling trajectory... 100%|████████████████████████

-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786868e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.837187087078686e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6
-> Truncation error given by norm of latest Qs matrix: 1.8371870870786864e-6


In [9]:
using LaTeXStrings
plot(θ_range, v_slopes, label="", fontfamily="Computer Modern", guidefontsize=15, tickfontsize=15, legendfontsize=15, border=:box, xlabel=L"\theta", ylabel=L"dF/dt")